# Extracting steps form workflow.md

In [57]:

#contains the workflow divided into {task : str ,steps : [str....]}
def get_workflow_dict(workflow_path):

    with open( workflow_path,"r") as f:
        workflow = f.read()

    wf={}
    wf["task"] = workflow.split("Workflow")[0]
    wf["workflow_steps"] = workflow.split("Workflow")[1].split("\n")
    return wf


workflow_path = "data/extracted_task_workflows/Workflow_3.md"
wf = get_workflow_dict(workflow_path)
wf

{'task': '---\nTask: 4. Add a new contact by filling out and submitting the form on the contacts add page\n',
 'workflow_steps': [':',
  'Step 1: Navigate to http://127.0.0.1:5000/dashboard',
  'Step 2: Click on the link that navigates to http://127.0.0.1:5000/contacts',
  'Step 3: Click on the link that navigates to http://127.0.0.1:5000/contacts/add',
  'Step 4: Fill out the form with the following details: ',
  '    - action: /contacts/add',
  '    - method: POST',
  '    - inputs: ',
  '        - name (text): the name of the new contact',
  '        - email (email): the email of the new contact',
  '        - notes (textarea): any notes about the new contact',
  'Step 5: Press the button to submit the form: ',
  '    - type: submit',
  '    - text: Save Contact',
  'Step 6: Verify that the new contact has been added successfully and the page has navigated to one of the following URLs: http://127.0.0.1:5000/dashboard, http://127.0.0.1:5000/contacts, http://127.0.0.1:5000/tickets, or

In [58]:
page = {}

# AGENT


### states and dataclasses

In [59]:
try:
    from langchain.tools import tool
except:
    !uv pip install langchain
# langgraph state variable (dict)

state = {
    "page" : None,  # Puppeteer page instance
    "home_url": None,
    "curr_url": None,
    "last_action": None,
    "history": [],
}

In [60]:
state

{'page': None,
 'home_url': None,
 'curr_url': None,
 'last_action': None,
 'history': []}

## TOOLS


In [61]:
try:
    import langgraph
except:
    !uv pip install langgraph

In [62]:
try:
    from pyppeteer import launch
except:
    !uv pip install pyppeteer

import asyncio

In [63]:
home_url = "http://127.0.0.1:5000/"
test_url = "http://127.0.0.1:5000/contacts/add"
 
browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])

page = await browser.newPage()
await page.goto(test_url, {'waitUntil': 'networkidle2'})
state['page'] = page


state["home_url"] = home_url
state["curr_url"] = page.url

In [64]:
import re


links = await page.evaluate('''() => 
                                       {
                                        const elements = Array.from(
                                            document.querySelectorAll(
                                                'a'
                                                
                                            ));   
                                        return elements.map(el => el.outerHTML);   
                                           
                                           
                                       }
                                       ''') 
    
    
# print(links)
extracted_links = []
for link in links:
    match = re.search(r'href=[\'"]([^\'"]+)[\'"]', link)

    if match:
        href_value = match.group(1)
        # print(href_value)
        
    
    extracted_links.append(href_value)





# buttons ,"\n", forms , " " , extracted_links

In [65]:
#click link tool

In [66]:
state["home_url"],extracted_links

('http://127.0.0.1:5000/',
 ['/dashboard', '/contacts', '/tickets', '/admin', '/contacts'])

In [67]:

@tool
async def navigate_to_link(state: dict, relative_url: str):
    """
    Navigate to a selected link (relative path like '/dashboard')
    Args:
        state (dict): Must contain a Puppeteer page object in `state["page"]`
        relative_url (str): e.g. '/dashboard'
    Returns:
        dict: Updated state with last action
    """
    page = state["page"]
    home_url = state["home_url"]

    # Construct absolute URL
    full_url = home_url.rstrip("/") + relative_url

    try:
        await page.goto(full_url, {'waitUntil': 'networkidle2'})
        state["curr_url"] = full_url
        state["last_action"] = f"✅ Navigated to {full_url}"
        return state
    except Exception as e:
        state["last_action"] = f"❌ Failed to navigate to {full_url}: {e}"
        return state



In [68]:
page

In [69]:
#click button tool

buttons = await page.evaluate('''() => 
                                       {
                                        const elements = Array.from(
                                            document.querySelectorAll(
                                             'button, input[type="button"], input[type="submit"], [onclick]'   
                                                
                                            )
                                            
                                        );   
                                        return elements.map(el => el.outerHTML);   
                                           
                                           
                                       }
                                       ''') 


buttons


['<button type="submit">Save Contact</button>']

In [70]:

from bs4 import BeautifulSoup


@tool
async def click_button_by_html(state: dict, html_string: str) -> dict:
    """
    Extract button text from HTML and click it on the current page.
    Args:
        state (dict): Must contain a Puppeteer page object in `state["page"]`
        html_string (str): e.g. '<button type="submit">Save Contact</button>'
    Returns:
        dict: Updated state with last action
    """
    page = state["page"]

    # Extract visible text from the button HTML
    try:
        soup = BeautifulSoup(html_string, 'html.parser')
        target_text = soup.get_text(strip=True)
    except Exception as e:
        state["last_action"] = f"❌ Failed to parse HTML: {e}"
        return state

    # Search all clickable elements
    try:
        buttons = await page.querySelectorAll('button, input[type="button"], input[type="submit"], [onclick]')
        
        for button in buttons:
            text = await page.evaluate('(el) => el.innerText || el.value || el.getAttribute("onclick") || ""', button)
            if text and target_text.lower() in text.strip().lower():
                await button.click()
                state["last_action"] = f"✅ Clicked button with text: {text.strip()}"
                return state
        
        state["last_action"] = f"❌ No button found with text: {target_text}"
        return state
    except Exception as e:
        state["last_action"] = f"❌ Error while clicking button: {e}"
        return state

# await click_button_by_html(state , buttons)


In [71]:
#fill form tool



forms = await page.evaluate('''() => 
                                       {
                                        const elements = Array.from(
                                            document.querySelectorAll(
                                                'form'
                                                
                                            )
                                            
                                        );   
                                        return elements.map(el => el.outerHTML);   
                                           
                                           
                                       }
                                       ''') 



In [72]:



from bs4 import BeautifulSoup

def parse_form_fields(html):
    soup = BeautifulSoup(html, "html.parser")
    inputs = soup.find_all(['input', 'textarea'])
    field_names = [tag.get("name") for tag in inputs if tag.get("name")]
    return field_names
    
# parse_form_fields(forms[0])

# use this to the the field values for prompt


In [73]:


values = {
    "name": "Manodeep Ray",
    "email": "manodeep@example.com",
    "notes": "College friend"
}


In [74]:
page.url


'http://127.0.0.1:5000/contacts/add'

In [ ]:
async def fill_form_fields(page, values: dict):
    """
    Fill out and submit a form using parsed field names and user-provided values.
    Args:
        page: Pyppeteer page object
        values: Dict of field names and their values
    """
    for field_name, field_value in values.items():

        # field_value = "<script>alert('XSS Attack!');</script>"


        selector = f'input[name="{field_name}"], textarea[name="{field_name}"], select[name="{field_name}"]'
        print(f"Filling: {selector} -> {field_value}")
        try:
            await page.waitForSelector(selector, timeout=2000)

            # Clear the input value using evaluate safely
            await page.evaluate(
                """(selector) => {
                    const el = document.querySelector(selector);
                    if (el) el.value = "";
                }""",
                selector
            )

            # Handle <select> separately if needed
            tag_name = await page.evaluate(
                """(selector) => {
                    const el = document.querySelector(selector);
                    return el ? el.tagName.toLowerCase() : null;
                }""",
                selector
            )

            if tag_name == "select":
                await page.select(selector, field_value)
            else:
                await page.type(selector, field_value)

        except Exception as e:
            print(f"⚠️ Failed to fill {field_name}: {e}")

    # Try to click the submit button
    try:
        submit_selectors = [
            'form button[type="submit"]',
            'form input[type="submit"]',
            'button[type="submit"]',
            'input[type="submit"]'
        ]
        for submit_selector in submit_selectors:
            try:
                await page.waitForSelector(submit_selector, timeout=1000)
                await page.click(submit_selector)
                await page.waitForNavigation(timeout=5000)
                return "✅ Form submitted."
            except:
                continue
        return "⚠️ No submit button found."
    except Exception as e:
        return f"⚠️ Failed to submit form: {e}"


In [110]:
@tool
async def form_filling_tool(state: dict, form_html: str, values: dict):
    """
    Fill out and submit a form using the parsed input names and user-provided values.
    
    Args:
        state: LangGraph agent state with a Puppeteer page
        form_html: HTML string of the <form>
        values: Dict of field names and their values (e.g., {"name": "John", "email": "john@example.com"})
    """
    fields = parse_form_fields(form_html)
    filtered_values = {k: v for k, v in values.items() if k in fields}
    
    result = await fill_form_fields(state["page"], filtered_values)
    state["last_action"] = result
    return state


state = await form_filling_tool.arun({
    "state": state,
    "form_html": form_html,
    "values": values
})


NameError: name 'form_html' is not defined

In [111]:
state

{'home_url': 'http://127.0.0.1:5000/',
 'page': <pyppeteer.page.Page at 0x7acce41ee8a0>,
 'page_html': '<!DOCTYPE html><html lang="en"><head>\n    <meta charset="UTF-8">\n    <title>MiniCRM</title>\n    <link rel="stylesheet" href="/static/style.css">\n</head>\n<body>\n    <header>\n        <h1>MiniCRM Dashboard</h1>\n        <nav>\n            <a href="/dashboard">🏠 Dashboard</a> |\n            <a href="/contacts">📇 Contacts</a> |\n            <a href="/tickets">🎫 Tickets</a> |\n            <a href="/admin">⚙️ Admin</a>\n        </nav>\n        <hr>\n    </header>\n\n    \n      \n    \n\n    <main>\n        \n<h2>➕ Add New Contact</h2>\n\n<form action="/contacts/add" method="post">\n    <label for="name">Name:</label><br>\n    <input type="text" name="name" required=""><br><br>\n\n    <label for="email">Email:</label><br>\n    <input type="email" name="email" required=""><br><br>\n\n    <label for="notes">Notes:</label><br>\n    <textarea name="notes" rows="4" cols="50"></textarea><b

In [112]:

# await navigate_to_link(state , relative_url)
# await click_button_by_html(state , buttons)
# await form_filling_tool(state , form_html , values )




tools = [
    navigate_to_link,
    click_button_by_html,
    form_filling_tool,
]

state['tools'] = tools

# creating the tools workflow

In [113]:
prompt = f"""
You are a workflow planner agent.

## Objective:
You are given:
1. A task to complete.
2. A list of natural language steps that must be followed in sequence.
3. A list of available tools (functions) in JSON format — each tool has a name and a description.

Your job:
- Match each step with the most appropriate tool.
- Describe the specific action and **arguments/parameters** the tool should use.
- Include details like which values are passed in (if relevant).
- if there is a form to fill do not have a step to click button to subimt form as filling it would submit it automaticaly
---

## Task:
{wf["task"]}

## Steps:
{wf["workflow_steps"]}

## Input Values (e.g. for forms):
{wf.get("values", {})}

## Available Tools (in JSON):
{state["tools"]}

---

## Output format:
Produce a **single-line string**, no newlines. For each step, use this format:

`StepDescription -> ToolName -> DetailedAction`

Where:
- `StepDescription` is a natural language summary of the step.
- `ToolName` is the name of the tool to use.
- `DetailedAction` explains exactly what the tool should do, including what arguments or values are used (if applicable).

Separate each step with ` || ` (double pipe). Do not add newlines or extra commentary.

---

## Example Output:
Navigate to the contact creation page -> navigate_to_link -> Go to relative URL '/contacts/add' from base URL || Fill out the form with name, email, and notes -> form_filling_tool -> Fill 'name' with 'John Doe', 'email' with 'john@example.com', and 'notes' with 'New lead from campaign' || Click the submit button -> click_button_by_html -> Find and click button with text 'Save Contact' and type='submit'

Only return the result in the required format.
"""


prompt

'\nYou are a workflow planner agent.\n\n## Objective:\nYou are given:\n1. A task to complete.\n2. A list of natural language steps that must be followed in sequence.\n3. A list of available tools (functions) in JSON format — each tool has a name and a description.\n\nYour job:\n- Match each step with the most appropriate tool.\n- Describe the specific action and **arguments/parameters** the tool should use.\n- Include details like which values are passed in (if relevant).\n- if there is a form to fill do not have a step to click button to subimt form as filling it would submit it automaticaly\n---\n\n## Task:\n---\nTask: 4. Add a new contact by filling out and submitting the form on the contacts add page\n\n\n## Steps:\n[\':\', \'Step 1: Navigate to http://127.0.0.1:5000/dashboard\', \'Step 2: Click on the link that navigates to http://127.0.0.1:5000/contacts\', \'Step 3: Click on the link that navigates to http://127.0.0.1:5000/contacts/add\', \'Step 4: Fill out the form with the foll

In [114]:
import llm_bridge

llm = llm_bridge.Bridge()



response = llm.get_response(prompt)

In [115]:
toolflow = response.choices[0].message.content


In [116]:
tool_flow_list = toolflow.split("||")
tool_flow_list

["Navigate to the dashboard page -> navigate_to_link -> Go to relative URL '/dashboard' from base URL ",
 " Navigate to the contacts page -> navigate_to_link -> Go to relative URL '/contacts' from base URL ",
 " Navigate to the contact creation page -> navigate_to_link -> Go to relative URL '/contacts/add' from base URL ",
 " Fill out the form with name, email, and notes -> form_filling_tool -> Fill 'name' with provided name, 'email' with provided email, and 'notes' with provided notes ",
 ' Verify the new contact has been added -> No tool available for this step as it requires verification, however, the form_filling_tool will submit the form and navigate to the next page automatically, achieving the same result as the original steps 5 and 6']

## from tool flow get the node graph required

In [117]:
tool_registry = {
    "navigate_to_link": navigate_to_link,
    "form_filling_tool": form_filling_tool,
    "click_button_by_html": click_button_by_html
}


In [118]:
example =  {
        "step": "Navigate to the contact creation page",
        "tool": "navigate_to_link",
        "action": "Go to relative URL '/contacts/add' from base URL"
    }

In [119]:
print(tool_flow_list)
parsed_plan = []
for s in tool_flow_list:
    parsed = {}

    match = re.match(r"^(.*?)\s*->", s)
    if match:
        step_description = match.group(1).strip()
        print(step_description)

    match = re.match(r"^[^>]*->\s*([^>]+?)\s*->", s)
    if match:
        tool_name = match.group(1).strip()
        print(tool_name)

    match = re.match(r"^[^>]*->[^>]*->\s*(.*)$", s)
    if match:
        action = match.group(1).strip()
        print(action)
        
    parsed['step'] = step_description
    parsed['tool'] = tool_name
    parsed['action'] = action

    parsed_plan.append(parsed)
    
    
    

["Navigate to the dashboard page -> navigate_to_link -> Go to relative URL '/dashboard' from base URL ", " Navigate to the contacts page -> navigate_to_link -> Go to relative URL '/contacts' from base URL ", " Navigate to the contact creation page -> navigate_to_link -> Go to relative URL '/contacts/add' from base URL ", " Fill out the form with name, email, and notes -> form_filling_tool -> Fill 'name' with provided name, 'email' with provided email, and 'notes' with provided notes ", ' Verify the new contact has been added -> No tool available for this step as it requires verification, however, the form_filling_tool will submit the form and navigate to the next page automatically, achieving the same result as the original steps 5 and 6']
Navigate to the dashboard page
navigate_to_link
Go to relative URL '/dashboard' from base URL
Navigate to the contacts page
navigate_to_link
Go to relative URL '/contacts' from base URL
Navigate to the contact creation page
navigate_to_link
Go to rel

In [120]:
parsed_plan

[{'step': 'Navigate to the dashboard page',
  'tool': 'navigate_to_link',
  'action': "Go to relative URL '/dashboard' from base URL"},
 {'step': 'Navigate to the contacts page',
  'tool': 'navigate_to_link',
  'action': "Go to relative URL '/contacts' from base URL"},
 {'step': 'Navigate to the contact creation page',
  'tool': 'navigate_to_link',
  'action': "Go to relative URL '/contacts/add' from base URL"},
 {'step': 'Fill out the form with name, email, and notes',
  'tool': 'form_filling_tool',
  'action': "Fill 'name' with provided name, 'email' with provided email, and 'notes' with provided notes"},
 {'step': 'Verify the new contact has been added',
  'tool': 'form_filling_tool',
  'action': "Fill 'name' with provided name, 'email' with provided email, and 'notes' with provided notes"}]

In [121]:
content = await page.content()
content

'<!DOCTYPE html><html lang="en"><head>\n    <meta charset="UTF-8">\n    <title>MiniCRM</title>\n    <link rel="stylesheet" href="/static/style.css">\n</head>\n<body>\n    <header>\n        <h1>MiniCRM Dashboard</h1>\n        <nav>\n            <a href="/dashboard">🏠 Dashboard</a> |\n            <a href="/contacts">📇 Contacts</a> |\n            <a href="/tickets">🎫 Tickets</a> |\n            <a href="/admin">⚙️ Admin</a>\n        </nav>\n        <hr>\n    </header>\n\n    \n      \n    \n\n    <main>\n        \n<h2>🔍 Search Results</h2>\n\n\n    <p>No matching contacts found.</p>\n\n\n<p><a href="/contacts">← Back to Contacts</a></p>\n\n    </main>\n\n</body></html>'

In [122]:
def get_json_from_output(text):
    match = re.search(r'```json\s*(\{.*?\})\s*```', text, re.DOTALL)

    if match:
        json_str = match.group(1)
        json_data = json.loads(json_str)
        # print(json_data)
    else:
        print("No JSON found.")
    
    return json_data

json_data = get_json_from_output(text)

NameError: name 'text' is not defined

In [123]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

# 1. Define your LLM
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./.env")


GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# Groq model setup (e.g., Mixtral)
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",  # or gemma-7b-it, llama3-8b-8192
    groq_api_key=GROQ_API_KEY # or set via env variable
)









# 2. Define the agent's prompt
prompt = PromptTemplate.from_template("""
You are an automation agent. Your job is to decide which tool to use and what values to fill in, based on the page's HTML and the current step.

Available tools:
- navigate_to_link
- form_filling_tool
- click_button_by_html

Each tool has its own input format:
- navigate_to_link: {{ "relative_url": "..." }}
- form_filling_tool: {{ "form_html": "...", "values": {{ "name": "...", "email": "...", "notes": "..." }} }}
- click_button_by_html: {{ "button_html": "..." }}

Reply with a JSON object:
{{ "tool": "...", "tool_args": {{ ... }} }}


only when filling form fill the empty values with some examples 


Step: {step_description}

Action for the agent to do : {step_action}
Current Page HTML:
{page_html}


- Choose the correct tool and provide only the **required minimal arguments**.
- Wrap the final output in **markdown-style fenced code block** with `json`, like this:
- fill the form values with dummy names

```json
```




""")






# 3. Wrap in LLMChain
agent = LLMChain(llm=llm, prompt=prompt)

# 4. Your function with the agent integrated
async def run_plan(parsed_plan, initial_page_html):
    state = {
        "home_url":"http://127.0.0.1:5000/",
        "page":page,
        "page_html": initial_page_html,
        "current_step": 0,
        "history": []
    }

    for idx, step_info in enumerate(parsed_plan):
        print(f"\n➡️ Step {idx+1}: {step_info['step']}")

        page_html = state["page_html"]
        step_description = step_info["step"]
        step_action =  step_info["action"]
        # 5. LLM agent input
        agent_input = {
            "page_html": page_html,
            "step_description": step_description,
            "step_action": step_action
            
        }

        # 6. Use LLM agent to choose tool and args
        # decision_raw = await agent.ainvoke(agent_input)
        # print(decision_raw)
        # decision = json.loads(decision_raw["text"]) 

        try:
            decision_raw = await agent.ainvoke(agent_input)
            print(f"AGENT OUTPUT : {decision_raw} \n\n")
             # assumes LLMChain returns text field
            
        except Exception as e:
            print(f"❌ Failed to get response from agent: {e}")
            break


        try:
            decision = get_json_from_output(decision_raw["text"]) 
            print(f" EXTRACTED JSON : {decision}")
        except Exception as e:
            print(f"❌ Failed to parse agent output: {e}")
        
        

        tool_name = decision.get("tool")
        tool_args = decision.get("tool_args", {})
        tool_args['state'] = state
        tool_func = tool_registry.get(tool_name)


        print(f"TOOL DETAILS : TOOL_NAME -> {tool_name} | TOOL_ARGS  -> {tool_args} | REGISTRY FUNC -> {tool_func} ")

        if tool_func is None:
            print(f"❌ Tool '{tool_name}' not found")
            break

        result = await tool_func.ainvoke(tool_args)
        new_page_html = result.get("new_page_html", page_html)
            
        
        
        
        try:
            print(f"🔧 Running tool: {tool_name}")
            
            result = await tool_func.ainvoke(tool_args)
            new_page_html = result.get("new_page_html", page_html)

        except Exception as e:
            print(f"❌ Error running tool: {e}")
            break

        # 7. Update state
        state["history"].append({
            "step": step_info["step"],
            "tool": tool_name,
            "args": tool_args,
            "result": result
        })
        state["page_html"] = new_page_html
        state["current_step"] += 1

    print("✅ All steps executed.")
    return state


In [124]:
state = await run_plan(parsed_plan, content)


➡️ Step 1: Navigate to the dashboard page
AGENT OUTPUT : {'page_html': '<!DOCTYPE html><html lang="en"><head>\n    <meta charset="UTF-8">\n    <title>MiniCRM</title>\n    <link rel="stylesheet" href="/static/style.css">\n</head>\n<body>\n    <header>\n        <h1>MiniCRM Dashboard</h1>\n        <nav>\n            <a href="/dashboard">🏠 Dashboard</a> |\n            <a href="/contacts">📇 Contacts</a> |\n            <a href="/tickets">🎫 Tickets</a> |\n            <a href="/admin">⚙️ Admin</a>\n        </nav>\n        <hr>\n    </header>\n\n    \n      \n    \n\n    <main>\n        \n<h2>🔍 Search Results</h2>\n\n\n    <p>No matching contacts found.</p>\n\n\n<p><a href="/contacts">← Back to Contacts</a></p>\n\n    </main>\n\n</body></html>', 'step_description': 'Navigate to the dashboard page', 'step_action': "Go to relative URL '/dashboard' from base URL", 'text': '```json\n{\n    "tool": "navigate_to_link",\n    "tool_args": {\n        "relative_url": "/dashboard"\n    }\n}\n```'} 


 EX

'navigate_to_link'

## EXECUTING THE WORkFLOW STEPS

: 

: 

### function for taking one step 

: 